In [2]:
import pandas as pd

df = pd.read_csv('./data/artists.csv')
df.head()

/var/folders/39/yqq6m0qn4rj4_j3d6xk0bv_h0000gn/T/ipykernel_10724/174998356.py:3: DtypeWarning: Columns (2,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/artists.csv')


,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


In [3]:
pd.set_option('display.max_colwidth', 1000)

In [4]:
# !pip install nltk

In [5]:
# import nltk
# nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sharlenetio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

allTagList = []

def clean(tags):
    tagList = tags.split(" ")
    tagSet = set()
    for tag in tagList:
        if tag != "":
            tagSet.add(lemmatizer.lemmatize(tag))
            allTagList.append(tag)
    
    return " ".join(tagSet)

df2 = df[df['ambiguous_artist'] == False]
df2 = df2[['tags_mb', 'artist_mb']]
df2['tags_mb'] = df2['tags_mb'].str.lower()
df2['artist_mb'] = df2['artist_mb'].str.lower()
df2 = df2.dropna(subset=['tags_mb'])
df2 = df2.drop_duplicates(subset=['artist_mb'])
df2 = df2[df2['tags_mb'].apply(lambda x: isinstance(x, str))]
df2['tags_mb'] = df2['tags_mb'].str.replace(' ', '')
df2['tags_mb'] = df2['tags_mb'].str.replace(';', ' ')
df2['tags_mb'] = df2['tags_mb'].str.replace(
    r'[^a-z0-9\s]',
    '',
    regex=True,
)
df2['tags_mb'] = df2['tags_mb'].apply(clean)
allTagList.sort()
print(allTagList[4000:5000])

print(df2.shape)
df2.head()

['adultcontemporary', 'adultcontemporary', 'adultcontemporary', 'adultcontemporary', 'adultcontemporary', 'adulterer', 'adultorientatedrock', 'adventista', 'adventista', 'adventure', 'adventure', 'adventureclubrecords', 'adventuresinthenationalgeographic', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventuretime', 'adventurous', 'advert', 'advert', 'advertising', 'advocate', 'adyhall', 'adyva', 'aec', 'aegis', 'aepoch', 'aeris', 'aestreladacano', 'aestus', 'aevolution', 'afeminado', 'afewunevenrhymes', 'affection', 'affectionpuppets', 'affiance', 'affirmation', 'affirmations', 'afflatus', 'afilkartist', 'afilkartist', 'afilkartist', 'afilkartist', 'afilkartist', 'afilkartist', 'afilkartist', '

,tags_mb,artist_mb
0,ambientpop postbritpop british rockandindie britannique uk pop pianorock rock chapel english alternativerock poprock pianopop parlophone britpop,coldplay
1,electronic artpop british melancholic nude artrock chamberpop experimentalrock britannique uk sacredcows britpop ctaitmieuxavant england grunge oxford parlophone rockandindie bootleg rock english alternativerock,radiohead
2,funkmetal rap popandchart funkrocktributo alternative 90 10 80 funkrock poprock crossover 00s raprock usa american metal dvd funk rock alternativerock,red hot chili peppers
3,hiphop barbadien electropop pop popandchart dancepop rnb reggae dancehall electrohouse barbadian barbade rehab barbados dance rihanna contemporaryrb popsoul rb,rihanna
4,american poprap boombap v midwesthiphop turkish hiphop rapper comedyrap hardcorehiphop horrorcore u eminem 8mile rap detroithiphop popandchart,eminem


In [7]:
df2 = df2[:20000]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df2['tags_mb'])

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [10]:
df3 = pd.DataFrame(similarities, columns=df2['artist_mb'], index=df2['artist_mb']).reset_index()

df3.head()

artist_mb,artist_mb,coldplay,radiohead,red hot chili peppers,rihanna,eminem,the killers,kanye west,nirvana,muse,...,u‐men,cosmic psychos,little fish,bart skils,papermoon,the thirst,mc conrad,heavy load,jean constantin,tommy keene
0,coldplay,1.000000,0.303926,0.061851,0.016395,0.000000,0.101358,0.025714,0.053596,0.213021,...,0.000000,0.0,0.062111,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,radiohead,0.303926,1.000000,0.029905,0.000000,0.000000,0.053311,0.000000,0.093177,0.170437,...,0.148797,0.0,0.269045,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,red hot chili peppers,0.061851,0.029905,1.000000,0.030501,0.086549,0.170898,0.024941,0.148863,0.154837,...,0.049980,0.0,0.000000,0.0,0.0,0.0,0.0,0.108319,0.0,0.0
3,rihanna,0.016395,0.000000,0.030501,1.000000,0.043319,0.000000,0.090716,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,eminem,0.000000,0.000000,0.086549,0.043319,1.000000,0.024524,0.264146,0.015901,0.000000,...,0.047811,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [12]:
input_artist = "coldplay"
recommendations = pd.DataFrame(df3.nlargest(11,input_artist)['artist_mb'])
recommendations = recommendations[recommendations['artist_mb']!=input_artist]
print(recommendations)

                          artist_mb
912                      starsailor
98                            keane
1403  the good, the bad & the queen
3527                  jarvis cocker
738                 vanessa carlton
71                          placebo
3579                      tom mcrae
723                       ben folds
2832                  the bluetones
615                           elbow
